In [1]:
import pandas as pd
from datetime import datetime
import re

Converting date columns to datetime format

In [4]:
def calculate_age(dob, event_time):
    age = event_time.year - dob.year - ((event_time.month, event_time.day) < (dob.month, dob.day))
    return age

def handle_special_age_encoding(age):
    if age > 300:  # Assuming that ages above 300 indicate ages above 89
        return 90
    else:
        return age

admissions_df = pd.read_csv('mimic-iii-clinical-database-1.4/ADMISSIONS.csv.gz')
patients_df = pd.read_csv('mimic-iii-clinical-database-1.4/PATIENTS.csv.gz')

admissions_df['ADMITTIME'] = pd.to_datetime(admissions_df['ADMITTIME'])
admissions_df['DISCHTIME'] = pd.to_datetime(admissions_df['DISCHTIME'])
admissions_df['DEATHTIME'] = pd.to_datetime(admissions_df['DEATHTIME'])

patients_df['DOB'] = pd.to_datetime(patients_df['DOB'])
patients_df['DOD'] = pd.to_datetime(patients_df['DOD'])

Merging 'ADMISSIONS' with 'PATIENTS' to associate DOB with each admission

In [5]:
admissions_with_dob = admissions_df.merge(patients_df[['SUBJECT_ID', 'DOB']], on='SUBJECT_ID', how='left')

Calculating age accordingly

In [6]:
admissions_with_dob['AGE_AT_ADMIT'] = admissions_with_dob.apply(lambda x: calculate_age(x['DOB'], x['ADMITTIME']), axis=1)

#special age encoding for patients above 89 years old
admissions_with_dob['AGE_AT_ADMIT'] = admissions_with_dob['AGE_AT_ADMIT'].apply(handle_special_age_encoding)

admissions_with_dob['AGE_AT_DEATH'] = admissions_with_dob.apply(lambda x: calculate_age(x['DOB'], x['DEATHTIME']) if pd.notnull(x['DEATHTIME']) else None, axis=1)
admissions_with_dob['AGE_AT_DEATH'] = admissions_with_dob['AGE_AT_DEATH'].apply(handle_special_age_encoding)

admissions_with_dob['AGE_AT_ADMIT'] = admissions_with_dob['AGE_AT_ADMIT'].clip(lower=0)

admissions_encoded = pd.get_dummies(admissions_with_dob, columns=['ADMISSION_TYPE', 'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY'], drop_first=True)

Q1 = admissions_encoded['AGE_AT_ADMIT'].quantile(0.25)
Q3 = admissions_encoded['AGE_AT_ADMIT'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = admissions_encoded[(admissions_encoded['AGE_AT_ADMIT'] < lower_bound) | (admissions_encoded['AGE_AT_ADMIT'] > upper_bound)]

outlier_count = outliers.shape[0]

admissions_encoded.to_csv('./cleaned_ADMISSIONS.csv', index=False)
patients_df.to_csv('./cleaned_PATIENTS.csv', index=False)

cleaning noteevents

In [ ]:
def clean_text(note):
    # I removed identified PHI markers (e.g., "[**...**]")
    note = re.sub(r'\[\*\*.*?\*\*\]', '', note)
    
    note = re.sub(r'\s+', ' ', note).strip()
    
    note = note.lower()
    
    return note

noteevents_df = pd.read_csv('mimic-iii-clinical-database-1.4/NOTEEVENTS.csv.gz')

noteevents_df['CLEANED_TEXT'] = noteevents_df['TEXT'].apply(clean_text)

noteevents_df.to_csv('mimic-iii-clinical-database-1.4/cleaned_NOTEEVENTS.csv', index=False)

cleaning prescriptions

In [ ]:
prescriptions_df = pd.read_csv('mimic-iii-clinical-database-1.4/PRESCRIPTIONS.csv.gz')

prescriptions_df['DRUG'] = prescriptions_df['DRUG'].str.strip().str.lower()

prescriptions_df['DOSE_VAL_RX'] = pd.to_numeric(prescriptions_df['DOSE_VAL_RX'], errors='coerce')

prescriptions_df = prescriptions_df.dropna(subset=['DOSE_VAL_RX'])

prescriptions_df['ROUTE'] = prescriptions_df['ROUTE'].str.strip().str.lower()

prescriptions_df['STARTDATE'] = pd.to_datetime(prescriptions_df['STARTDATE'], errors='coerce')
prescriptions_df['ENDDATE'] = pd.to_datetime(prescriptions_df['ENDDATE'], errors='coerce')

prescriptions_df = prescriptions_df[(prescriptions_df['STARTDATE'].dt.year > 1900) & (prescriptions_df['STARTDATE'].dt.year < datetime.now().year)]
prescriptions_df = prescriptions_df[(prescriptions_df['ENDDATE'].dt.year > 1900) & (prescriptions_df['ENDDATE'].dt.year < datetime.now().year)]

prescriptions_df['ROUTE'].fillna('unknown', inplace=True)

prescriptions_df.to_csv('./cleaned_PRESCRIPTIONS.csv', index=False)

cleaning diagnoses_icd

In [8]:
diagnoses_icd_df = pd.read_csv('mimic-iii-clinical-database-1.4/D_ICD_DIAGNOSES.csv.gz')

diagnoses_icd_df['LONG_TITLE'] = diagnoses_icd_df['LONG_TITLE'].str.strip().str.lower()
diagnoses_icd_df['SHORT_TITLE'] = diagnoses_icd_df['SHORT_TITLE'].str.strip().str.lower()

diagnoses_icd_df.dropna(subset=['LONG_TITLE'], inplace=True)

diagnoses_icd_df.drop_duplicates(inplace=True)

diagnoses_icd_df.to_csv('./cleaned_D_ICD_DIAGNOSES.csv', index=False)

cleaning icu_stays

In [10]:
icustays_df = pd.read_csv('mimic-iii-clinical-database-1.4/ICUSTAYS.csv.gz')
#converting to datetime format
icustays_df['INTIME'] = pd.to_datetime(icustays_df['INTIME'])
icustays_df['OUTTIME'] = pd.to_datetime(icustays_df['OUTTIME'])

icustays_df['LOS'] = (icustays_df['OUTTIME'] - icustays_df['INTIME']).dt.total_seconds() / (24 * 60 * 60)

invalid_stays = icustays_df[icustays_df['INTIME'] > icustays_df['OUTTIME']]

icustays_df = icustays_df.drop(invalid_stays.index)

icustays_df.dropna(subset=['INTIME', 'OUTTIME'], inplace=True)

duplicates = icustays_df[icustays_df.duplicated(subset=['ICUSTAY_ID'], keep=False)]

icustays_df.to_csv('./cleaned_ICUSTAYS.csv', index=False)